# Convolutional Neural Network

### Importing the libraries

In [0]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [0]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [0]:
# Create an ImageDataGenerator object for training data (augmentation step)
# This will preprocess and augment the images to improve model generalization
train_datagen = ImageDataGenerator(
    rescale = 1./255,         # Rescales pixel values from 0–255 to 0–1 (normalization)
    shear_range = 0.2,        # Randomly applies shear transformations (slanting the image), Helps the model become invariant to small distortions or perspective changes.
    zoom_range = 0.2,         # Randomly zooms inside pictures
    horizontal_flip = True    # Randomly flips some images horizontally (left-right)
)

# Load and preprocess the training images from the specified directory
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',   # Path to the folder containing class subfolders (e.g., 'cats' and 'dogs')
    target_size = (64, 64),   # Resize all images to 64x64 pixels
    batch_size = 32,          # Process images in batches of 32
    class_mode = 'binary'     # Since this is a binary classification (2 classes), use 'binary' mode
)


### Preprocessing the Test set

In [0]:
# only rescale for test data, We do not augment test data — we want to evaluate the model on real, untouched examples.
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [0]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# filters=32 -- This layer will learn 32 different features from the input image.
# kernel_size=3 -- Size of each filter (3×3 in this case). It slides over the image to capture local patterns.
# input_shape=[64, 64, 3] Shape of the input images: 64×64 pixels, 3 color channels (RGB). This is required only for the first layer.
# this layer will learn spatial patterns in the input image (like edges, textures, etc.).

### Step 2 - Pooling

In [0]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#  This layer helps to downsample the feature maps, reducing their size while keeping the most important information.
# pool_size=2	Size of the pooling window (2x2). It looks at non-overlapping 2×2 sections of the input.
# strides=2	The step size. A stride of 2 moves the window 2 pixels at a time — effectively halving the spatial dimensions (height & width).


### Adding a second convolutional layer

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Layers
# Conv2D → MaxPool2D → Conv2D → MaxPool2D → Flatten → Dense


### Step 3 - Flattening

In [0]:
    cnn.add(tf.keras.layers.Flatten())
    
    #The Flatten() layer converts the 2D (or 3D) output from the previous layer (usually a set of feature maps) into a 1D vector.

### Step 4 - Full Connection

In [0]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Dense layer: Every neuron in this layer is connected to all the neurons in the previous layer.
#units=128: This layer has 128 neurons (also called nodes or units). It controls the model’s capacity to learn complex patterns.
#activation='relu': Applies the Rectified Linear Unit (ReLU) activation function, which introduces non-linearity. It outputs max(0, x), so all negative values become zero.

### Step 5 - Output Layer

In [0]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#units=1: The layer has 1 neuron because the model predicts a single output (e.g., class 0 or class 1).
#activation='sigmoid': Uses the sigmoid activation function, which outputs a value between 0 and 1, representing the probability of belonging to class 1.

## Part 3 - Training the CNN

### Compiling the CNN

In [0]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [0]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# Convert the image to a numpy array (height x width x channels)
test_image = image.img_to_array(test_image)
# Add an extra dimension at axis 0 to simulate batch size of 1
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [0]:
print(prediction)